# a. SQL Databases

# b. NoSQL Databases

# c. SQL or NoSQL

# d. Python Database Frameworks

# e. Database Management with Flask-SQLAlchemy

```shell
pip install flask-sqlalchemy
```

In [1]:
import os
from flask import Flask, render_template, session, redirect, url_for
from flask_bootstrap import Bootstrap
from flask_moment import Moment
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import DataRequired

# NEW
from flask_sqlalchemy import SQLAlchemy

In [2]:
app = Flask(__name__)
app.config['SECRET_KEY'] = 'hard to guess string'
app.config['SERVER_NAME'] = 'floydjjluo.me'
#app.config['SERVER_NAME'] = ''
app.config['WTF_CSRF_ENABLED'] = False


# basedir = os.path.abspath(os.path.dirname(__file__)); print(basedir)
basedir = ''
DB_URI  = 'sqlite:///' + os.path.join(basedir, 'data.sqlite');print(DB_URI)
app.config['SQLALCHEMY_DATABASE_URI'] = DB_URI
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False


sqlite:///data.sqlite


In [3]:
bootstrap = Bootstrap(app)
moment = Moment(app)

# new
db = SQLAlchemy(app)


# f. Model Definition

In [4]:
# basic the same as sqlalchemy

class Role(db.Model):
    __tablename__ = 'roles'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), unique=True)

    def __repr__(self):
        return '<Role %r>' % self.name


class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(64), unique=True, index=True)
    role_id = db.Column(db.Integer, db.ForeignKey('roles.id'))

    def __repr__(self):
        return '<User %r>' % self.username


# g. Relationships

In [1]:
import os
from flask import Flask, render_template, session, redirect, url_for
from flask_bootstrap import Bootstrap
from flask_moment import Moment
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import DataRequired

# NEW
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SECRET_KEY'] = 'hard to guess string'
app.config['SERVER_NAME'] = 'floydjjluo.me'
#app.config['SERVER_NAME'] = ''
app.config['WTF_CSRF_ENABLED'] = False


# basedir = os.path.abspath(os.path.dirname(__file__)); print(basedir)
basedir = ''
DB_URI  = 'sqlite:///' + os.path.join(basedir, 'data.sqlite');print(DB_URI)
app.config['SQLALCHEMY_DATABASE_URI'] = DB_URI
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

bootstrap = Bootstrap(app)
moment = Moment(app)
db = SQLAlchemy(app)

sqlite:///data.sqlite


In [2]:
# basic the same as sqlalchemy

class Role(db.Model):
    __tablename__ = 'roles'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), unique=True)
    
    users = db.relationship('User', backref = 'role') # new 

    def __repr__(self):
        return '<Role %r>' % self.name


class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(64), unique=True, index=True)
    
    role_id = db.Column(db.Integer, db.ForeignKey('roles.id')) # new

    def __repr__(self):
        return '<User %r>' % self.username


# h. Database Operations

### 8.1 Creating the Tables

In [5]:
# from hello import db
db.create_all()

In [6]:
db.drop_all()
db.create_all()

### 8.2 Inserting Rows

In [7]:
admin_role = Role(name = 'Admin')
mod_role = Role(name = 'Moderator')
user_role = Role(name = 'User')

In [9]:
Floyd = User(username = 'Floyd', role = admin_role)

Susan = User(username = 'Susan', role = user_role)
David = User(username = 'David', role = user_role)

In [11]:
print(admin_role.id)

None


In [12]:
db.session.add(admin_role)
db.session.add(mod_role)

db.session.add(user_role)

db.session.add(Floyd)

db.session.add(Susan)


db.session.add(David)

In [13]:
db.session.commit()

In [14]:
admin_role.id

1

In [15]:
user_role.id

3

In [16]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')

pd.read_sql("SELECT * FROM Users", conn)

,id,username,role_id
0,1,Floyd,1
1,2,Susan,3
2,3,David,3


In [17]:
pd.read_sql("SELECT * FROM Roles", conn)

,id,name
0,1,Admin
1,2,Moderator
2,3,User


### 8.3 Modifying Rows

In [18]:
admin_role.name

'Admin'

In [19]:
admin_role.name = 'Administrator'

In [20]:
db.session.add(admin_role)
db.session.commit()

In [21]:
pd.read_sql("SELECT * FROM Roles", conn)

,id,name
0,1,Administrator
1,2,Moderator
2,3,User


### 8.4 Deleting Rows

In [22]:
db.session.delete(mod_role)
db.session.commit()

In [23]:
pd.read_sql("SELECT * FROM Roles", conn)

,id,name
0,1,Administrator
1,3,User


### 8.5 Querying Rows

In [24]:
Role.query.all()

[<Role 'Administrator'>, <Role 'User'>]

In [25]:
User.query.all()

[<User 'Floyd'>, <User 'Susan'>, <User 'David'>]

In [27]:
User.query.filter_by(role = user_role).all()

[<User 'Susan'>, <User 'David'>]

In [30]:
print(str(User.query.filter_by(role = user_role)))

SELECT users.id AS users_id, users.username AS users_username, users.role_id AS users_role_id 
FROM users 
WHERE ? = users.role_id


In [31]:
User.query.filter_by(role = user_role).first()

<User 'Susan'>

In [33]:
users = user_role.users
users

[<User 'Susan'>, <User 'David'>]

In [34]:
Susan = users[0]
Susan.role

<Role 'User'>

**Change the lazy to dynamic**

In [1]:
import os
from flask import Flask, render_template, session, redirect, url_for
from flask_bootstrap import Bootstrap
from flask_moment import Moment
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import DataRequired

# NEW
from flask_sqlalchemy import SQLAlchemy
import sqlite3
import pandas as pd


app = Flask(__name__)
app.config['SECRET_KEY'] = 'hard to guess string'
app.config['SERVER_NAME'] = 'floydjjluo.me'
#app.config['SERVER_NAME'] = ''
app.config['WTF_CSRF_ENABLED'] = False


# basedir = os.path.abspath(os.path.dirname(__file__)); print(basedir)
basedir = ''
DB_URI  = 'sqlite:///' + os.path.join(basedir, 'data.sqlite');print(DB_URI)
app.config['SQLALCHEMY_DATABASE_URI'] = DB_URI
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

bootstrap = Bootstrap(app)
moment = Moment(app)
db = SQLAlchemy(app)

sqlite:///data.sqlite


In [2]:
# basic the same as sqlalchemy

class Role(db.Model):
    __tablename__ = 'roles'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), unique=True)
    
    users = db.relationship('User', backref = 'role', lazy = 'dynamic') # new 

    def __repr__(self):
        return '<Role %r>' % self.name


class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(64), unique=True, index=True)
    
    role_id = db.Column(db.Integer, db.ForeignKey('roles.id')) 

    def __repr__(self):
        return '<User %r>' % self.username



In [3]:
db.create_all()
admin_role = Role(name = 'Admin')
mod_role = Role(name = 'Moderator')
user_role = Role(name = 'User')
Floyd = User(username = 'Floyd', role = admin_role)

Susan = User(username = 'Susan', role = user_role)
David = User(username = 'David', role = user_role)


db.session.add(admin_role)
db.session.add(mod_role)
db.session.add(user_role)
db.session.add(Floyd)
db.session.add(Susan)
db.session.add(David)
db.session.commit()

In [4]:
conn = sqlite3.connect('data.sqlite')

pd.read_sql("SELECT * FROM Users", conn)

,id,username,role_id
0,1,Floyd,1
1,2,Susan,3
2,3,David,3


In [5]:
pd.read_sql("SELECT * FROM Roles", conn)

,id,name
0,1,Admin
1,2,Moderator
2,3,User


In [6]:
user_role.users # this is different from last section

In [7]:
user_role.users.count()

2

In [8]:
user_role.users.order_by(User.username).all()

[<User 'David'>, <User 'Susan'>]

# i. Database Use in View Function

In [9]:
from IPython.core.display import display, HTML

def show(string):
    display(HTML(string))

In [10]:
class NameForm(FlaskForm):
    name = StringField('What is your name?', validators=[DataRequired()])
    submit = SubmitField('Submit')

In [11]:
%%writefile templates/i_index.html
{% extends "base.html" %}
{% import "bootstrap/wtf.html" as wtf %}

{% block title %}Flasky{% endblock %}

{% block page_content %}
<div class="page-header">
    <h1>Hello, {% if name %}{{ name }}{% else %}Stranger{% endif %}!</h1>
    {% if not known %}
    <p>Pleased to meet you!</p>
    {% else %}
    <p>Happy to see you again!</p>
    {% endif %}
</div>
{{ wtf.quick_form(form) }}
{% endblock %}

Overwriting templates/i_index.html


In [12]:
app_ctx = app.app_context()
app_ctx.push()
client = app.test_client(use_cookies=True)

In [13]:
@app.route('/i_index', methods=['GET', 'POST'])
def i_index():
    form = NameForm()
    
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.name.data).first()
        if user is None:
            user = User(username=form.name.data)
            db.session.add(user)
            db.session.commit()
            session['known'] = False
        else:
            session['known'] = True
        session['name'] = form.name.data
        
        # return redirect(url_for('index'))
        return client.get('/i_index')
    
    o = render_template('i_index.html', 
                        form=form, 
                        name=session.get('name'),
                        known=session.get('known', False)); show(o)
    return o

In [14]:
client.get('/i_index')

<Response streamed [200 OK]>

In [15]:
client.post('/i_index')

<Response streamed [200 OK]>

In [16]:
client.post('/i_index', data = {'name': 'Lloyd'})

<Response streamed [200 OK]>

In [17]:
pd.read_sql("SELECT * FROM Users", conn)

,id,username,role_id
0,1,Floyd,1.0
1,2,Susan,3.0
2,3,David,3.0
3,4,Lloyd,NaN


In [18]:
client.get('/i_index')

<Response streamed [200 OK]>

In [19]:
client.post('/i_index', data = {'name': 'Lloyd'})

<Response streamed [200 OK]>

In [20]:
client.get('/i_index')

<Response streamed [200 OK]>

In [21]:
client.post('/i_index', data = {'name': 'Floyd'})

<Response streamed [200 OK]>

In [22]:
client.get('/i_index')

<Response streamed [200 OK]>

# j. Database Migrations with Flask-Migrate

### 10. 1 Creating a Migration Repository

In [23]:
import os
from flask import Flask, render_template, session, redirect, url_for
from flask_bootstrap import Bootstrap
from flask_moment import Moment
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import DataRequired


from flask_sqlalchemy import SQLAlchemy


from flask_migrate import Migrate # NEW

import sqlite3
import pandas as pd


app = Flask(__name__)
app.config['SECRET_KEY'] = 'hard to guess string'
app.config['SERVER_NAME'] = 'floydjjluo.me'
#app.config['SERVER_NAME'] = ''
app.config['WTF_CSRF_ENABLED'] = False


# basedir = os.path.abspath(os.path.dirname(__file__)); print(basedir)
basedir = ''
DB_URI  = 'sqlite:///' + os.path.join(basedir, 'data.sqlite');print(DB_URI)
app.config['SQLALCHEMY_DATABASE_URI'] = DB_URI
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

bootstrap = Bootstrap(app)
moment = Moment(app)
db = SQLAlchemy(app)
migrate = Migrate(app, db)

sqlite:///data.sqlite


```shell
flask db init
```

### 10. 2 Creating a Migration Script

```shell
flask db migrate -m 'initial migration'
```

### 10.3 Upgrading the Database

```shell
flask db upgrade
```